In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
sns.set()

In [2]:
def Stinehart_Hart(x,a,b,c,d):
    constant = -273.15
    return 1/(a+b*np.log(x)+c*np.log(x)**2+d*np.log(x)**3) + constant
def Shockley(x,I,t)
    # fit for saturation current and temperature
    q = 1.6*10**(-19)  # electric charge in Coloumbs
    k = 1.3807*10**(-23) # Boltzman Constant in Joules
    
def chi_square_red(x, y, yerr, modelfunc, p_opt):
    numerator = (y - modelfunc(x, *p_opt)) ** 2
    denominator = yerr ** 2
    chi = sum(numerator / denominator)
    chired = chi / (len(y) - len(p_opt))
    return chired
def fitting(function,xdata,ydata,  guess,sigma = 0):
    if type(sigma) is int:
        fit_opt, fit_cov = curve_fit(function,xdata,ydata, p0=guess)
    else:
        fit_opt, fit_cov = curve_fit(function,xdata,ydata, p0=guess, sigma = sigma, absolute_sigma = True)
    return function(xdata,*fit_opt), fit_opt, fit_cov 


In [ ]:
thermo_data = np.loadtxt('thermo.csv', delimiter=',', usecols=(0,1), unpack=True)

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(thermo_data[0],thermo_data[1] )
init = (-90,0.3)
fit = fitting(Stinehart_Hart,thermo_data[0],thermo_data[1] ,guess =init)
plt.plot(thermo_data[0], fit[0], c = 'r')
plt.title('Hartman Relation Graph')
plt.ylabel('Scale Reading')
plt.xlabel('wavelength (nm)')

In [ ]:
diode_data = np.loadtxt('diode.csv', delimiter=',', usecols=(0,1), unpack=True)

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(thermo_data[0],thermo_data[1] )
init = (-90,0.3)
fit = fitting(Stinehart_Hart,thermo_data[0],thermo_data[1] ,guess =init)
plt.plot(thermo_data[0], fit[0], c = 'r')
plt.title('Hartman Relation Graph')
plt.ylabel('Scale Reading')
plt.xlabel('wavelength (nm)')